## Import Packages

In [1]:
import json
import pandas as pd
import random
import numpy as np
from tensorflow import keras
import nltk
from nltk.stem.lancaster import LancasterStemmer
# nltk.download('punkt')
stemmer = LancasterStemmer()

## Load data from intents.json
1. Get all words for preparing bag of words (BOW).
2. Get all intents for checking prediction results.
3. words and intents are in sorted order as model training process.

In [2]:
with open('intents.json') as f:
        data = json.load(f)

words = []  # store all words in data
intents = []  # store classes of intents
words_to_ignore = ['?', '.']

for intent in data['intents']:
    for pattern in intent['patterns']:
        words_each_pattern = nltk.word_tokenize(pattern)  # split the sentence
        words.extend(words_each_pattern)  # w is a list

    if intent['tag'] not in intents:
        intents.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in words_to_ignore]

words = sorted(list(set(words)))
intents = sorted(list(set(intents)))

## Load saved model

In [3]:
model = keras.models.load_model('model.h5')

## chat function for utilizing the model
### Parameter: sentence(string)

In [4]:
def chat(sentence):
    # Prepare bag of words:
    bag = []

    words_in_sentence = nltk.word_tokenize(sentence)
    words_in_sentence = [stemmer.stem(w.lower()) for w in words_in_sentence]

    for w in words:
        if w in words_in_sentence:
            bag.append(1)
        else:
            bag.append(0)
    
    # Format bag for trained model
    bag = pd.DataFrame([bag])

    # Make predictions
    results = model.predict(bag)[0]
    results_index = np.argmax(results)  # Choose one with highest probability

    intent = intents[results_index]

    if results[results_index] > 0.8:
        for tg in data["intents"]:
            if tg['tag'] == intent:
                responses = tg['responses']
        print(random.choice(responses))  # Randomly choose one response
    else:
        print("I didnt understander. Try again")

In [5]:
chat('is anyone here?')

Hello!


In [6]:
chat('how can i call you?')

I'm Justin.


In [7]:
chat('can you tell me something about yourself?')

I am a fast and self-motivated learner.


In [8]:
chat('I want to know your education background')

I obtained a BEng in Mechanical and Automation Engineering from CUHK.
